参照サイト：
http://www.eiyo.ac.jp/recipe/
※以下にアクセス（レシピ一覧）
http://www.eiyo.ac.jp/recipe/recipe.html

【目次】
1セル：ここ
2セル：ライブラリのインポート
3セル：関数指定
4セル：変数指定
5セル～：メイン（処理実行）


In [1]:
from urllib.parse import quote
import re
import requests
from bs4 import BeautifulSoup
from collections import OrderedDict
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.options import Options

import pandas as pd

import xlsxwriter

import os

#サイトから再取得
# def get_current_item_dict():

# エクセルへエクスポート
# df['df_cuisine_list'].to_excel('C:/Users/Takayoshi_Hoshino/20180602_発表_omameさん/料理リスト_材料以外.xlsx')
# df['df_cuisine_list_for_ingredients'].to_excel('C:/Users/Takayoshi_Hoshino/20180602_発表_omameさん/料理リスト_材料.xlsx')


In [2]:
#■関数

#女子栄養大学から料理の情報を取得
#引数：料理カテゴリのリスト（'主食' , '主菜' , '副菜' , '汁もの' , 'その他'）
#戻り値：（辞書）success：真偽値（最後までアクセス成功したか）、cuisine_list：各料理の情報辞書を格納したリスト（エクセル資料参照）
def get_cuisine_list(list_taisho_cuisine_cate_key):
    #レシピ一覧へアクセス失敗の場合、False
    success = True
    
    #料理リスト
    cuisine_list = []

    #抽出する料理カテゴリ（主菜、副菜など）
    #list_taisho_cuisine_cate_key = ['主菜' , '副菜']

    #料理カテゴリ（主菜、副菜など）の全リスト（データは各商品情報につくカテゴリの画像URL）
    all_cuisine_category_dict = {'主食':'/recipe/img/recipe/explanation_05.png'
                            ,'主菜':'/recipe/img/recipe/explanation_06.png'
                            ,'副菜':'/recipe/img/recipe/explanation_07.png'
                            ,'汁もの':'/recipe/img/recipe/explanation_08.png'
                            ,'その他':'/recipe/img/recipe/explanation_09.png'}

    #上記の辞書の値とキーを逆にする
    all_cuisine_category_dict_inv = {v:k for k , v in all_cuisine_category_dict.items()}

    url_recipe_top = 'http://www.eiyo.ac.jp/recipe/recipe.html'

    #参照するページ（次頁があれば情報取得後、この変数を更新する。次頁がなければ None）
    url_recipe_current = url_recipe_top

    #繰り返し処理（頁単位）
    while url_recipe_current != None:

        res_info = requests.get(url_recipe_current)

        # 正常に情報取得できた場合
        if str(res_info.status_code)[0] == '2':
            #商品一覧箇所（最初のdivは上部にある「レシピサイト」の画像なので、省く）
            bs_item_list = BeautifulSoup(res_info.text , 'html.parser').find_all('div' , class_='recipe')[1:]

            #各商品情報を繰り返し抽出（対象の料理カテゴリ以外はとばす）
            for div_item in bs_item_list:
                #料理カテゴリの画像URL
                url_category = div_item.find('p' , class_='first').find('img').get('src')

                #料理カテゴリ
                category = all_cuisine_category_dict_inv[url_category]

                #対象の料理カテゴリ以外はとばす
                if category not in list_taisho_cuisine_cate_key:
                    continue

                #料理名
                cuisine_name = div_item.find('div' , class_='top').find('p').text.strip()

                #画像(URL)
                #image = 'http://www.eiyo.ac.jp' + div_item.find('div' , class_='left').find('img').get('src').strip()

                #詳細サイトのURL
                url_item_detail = 'http://www.eiyo.ac.jp' + div_item.find('a').get('href')

                #詳細画面情報
                res_info_detail = requests.get(url_item_detail)

                #詳細画面へアクセスできたら取得・商品リストへ追加
                if str(res_info_detail.status_code)[0] == '2':
                    #bs化
                    bs_detail = BeautifulSoup(res_info_detail.text , 'html.parser')

                    #画像(URL)
                    image = 'http://www.eiyo.ac.jp' + bs_detail.find('div' , class_='photo').find('img').get('src')

                    #調理時間、各栄養のspanタグ
                    span_calo_salt_time = bs_detail.find_all('div' , class_='details')[1].find_all('span' , class_='right')

                    #調理時間
                    cooking_time = float(span_calo_salt_time[2].text.replace('分' , '').strip())            

                    #■栄養素取得-----------------------------------------------------------------------------

                    #カロリー
                    calorie = float(span_calo_salt_time[0].text.replace('kcal' , '').strip())

                    #塩分
                    salt = float(span_calo_salt_time[1].text.replace('g' , '').strip())

                    #タンパク質
                    protein = float(span_calo_salt_time[4].text.replace('g' , '').strip())

                    #脂質
                    lipid = float(span_calo_salt_time[5].text.replace('g' , '').strip())

                    #炭水化物
                    carbohydrate = float(span_calo_salt_time[6].text.replace('g' , '').strip())

                    #食物繊維
                    dietary_fiber = float(span_calo_salt_time[7].text.replace('g' , '').strip())

                    #鉄分
                    iron = float(span_calo_salt_time[8].text.replace('mg' , '').strip())

                    #カルシウム
                    calcium = float(span_calo_salt_time[9].text.replace('mg' , '').strip())

                    #栄養の辞書作成
                    nutrition = OrderedDict((('calorie' , calorie)
                                                ,('salt' , salt)
                                                ,('protein',protein)
                                                ,('lipid',lipid)
                                                ,('carbohydrate',carbohydrate)
                                                ,('dietary_fiber',dietary_fiber)
                                                ,('iron',iron)
                                                ,('calcium',calcium)))

                    #-----------------------------------------------------------------------------

                    #■材料・分量取得-----------------------------------------------------------------------------

                    #材料のspanタグ（ヘッダーを除く）
                    tr_ingredients = bs_detail.find('div' , id='ingredients').find_all('tr')[1:]

                    #材料の辞書
                    ingredients = {}

                    #材料情報を取得
                    for tr_tag in tr_ingredients:
                        #class=group parent はとばす（例：http://www.eiyo.ac.jp/recipe/recipe/view/id:175.html）
                        if tr_tag.get('class') == ['group', 'parent'] :
                            continue

                        ingredient_name = tr_tag.find('td' , class_='name').text.strip()
                        ingredient_quantity = tr_tag.find('td' , class_='quantity').text.strip()

                        #辞書へ追加
                        ingredients[ingredient_name] = ingredient_quantity


                    #-----------------------------------------------------------------------------
                    #作り方
                    how_to_cook = bs_detail.find('div' , id='directions').find('div' , class_='content').text.strip()

                    #料理リストに追加する辞書作成
                    cuisine_dict_for_list = OrderedDict((
                                                ('cuisine_name' , cuisine_name)
                                                ,('image' , image)
                                                ,('category' , category)
                                                ,('cooking_time' , cooking_time)
                                                ,('how_to_cook' , how_to_cook)
                                                ,('ingredients' , ingredients)
                                                ,('nutrition' , nutrition)))

                    #料理リストに追加
                    cuisine_list.append(cuisine_dict_for_list)

            #次頁ボタン（次がなければ aタグが無い）
            bs_next_page_footer = BeautifulSoup(res_info.text , 'html.parser').find_all('div' , class_='paginator')[1].find('span' , class_='next').find('a')

            #次頁ボタンがあればリンクを取得。なければ None
            if bs_next_page_footer != None:
                url_recipe_current = 'http://www.eiyo.ac.jp' + bs_next_page_footer.get('href')
            else:
                url_recipe_current = None

        else:
            success = False
            break
            
    return {'success':success , 'cuisine_list':cuisine_list}


#関数get_cuisine_listで取得したcuisine_listの情報をデータフレームにまとめる
#引数：料理リスト　※get_cuisine_listの戻り値（辞書）のキー「cuisine_list」の値
#戻り値：（辞書）df_cuisine_list：材料以外のデータフレーム　、df_cuisine_list_for_ingredients：材料以外のデータフレーム
def create_df_cuisine_list(cuisine_list):
    #材料以外のデータフレーム
    df_cuisine_list = pd.DataFrame()

    #No.、料理名、材料名、分量のデータフレーム
    df_cuisine_list_for_ingredients = pd.DataFrame()

    #No.（リスト番号）
    df_cuisine_list['No.'] = pd.Series([i for i in range(len(cuisine_list))])

    #No.、料理名、材料名、分量のデータフレーム用（行）
    row_num = 0

    for no , cuisine in enumerate(cuisine_list): 
        #材料以外のデータフレーム
        df_cuisine_list.loc[no , 'No.'] = no

        for key , val in cuisine.items():
            if key == 'nutrition':
                dict_nutrition = cuisine[key]
                for key_nutrition , val_nutrition in dict_nutrition.items():
                    df_cuisine_list.loc[no , key_nutrition] = val_nutrition
            elif key == 'ingredients':
                pass
            else :
                df_cuisine_list.loc[no , key] = val

        #No.、料理名、材料名、分量のデータフレーム
        for key_ingredient , val_ingredient in cuisine['ingredients'].items():
            df_cuisine_list_for_ingredients.loc[row_num , 'No.'] = no
            df_cuisine_list_for_ingredients.loc[row_num , 'category'] = cuisine['category']
            df_cuisine_list_for_ingredients.loc[row_num , 'cuisine_name'] = cuisine['cuisine_name']
            df_cuisine_list_for_ingredients.loc[row_num , '材料名'] = key_ingredient
            df_cuisine_list_for_ingredients.loc[row_num , '分量'] = val_ingredient

    #         print('---------------------------')
    #         print('row_num:' , row_num)
    #         print('No.:' , no)
    #         print('cuisine_name:' , cuisine['cuisine_name'])
    #         print('材料名' , key_ingredient)
    #         print('分量' , val_ingredient)
    #         print('---------------------------')

            row_num += 1

    return {'df_cuisine_list':df_cuisine_list , 'df_cuisine_list_for_ingredients':df_cuisine_list_for_ingredients}

In [3]:
#■変数

#各栄養の単位
unit_nutrition_list = OrderedDict((('calorie' , 'kcal')
                                    ,('salt' , 'g')
                                    ,('protein' , 'g')
                                    ,('lipid' , 'g')
                                    ,('carbohydrate' , 'g')
                                    ,('dietary_fiber' , 'g')
                                    ,('iron' , 'mg')
                                    ,('calcium' , 'mg')))

#抽出する料理カテゴリ（選択肢：'主食' , '主菜' , '副菜' , '汁もの' , 'その他'）　※適宜変更
list_taisho_cuisine_cate_key = ['主食' , '主菜' , '副菜' , '汁もの' , 'その他']



In [4]:
#■メイン（処理実行）

#サイトからデータ取得（取得する料理カテゴリ「list_taisho_cuisine_cate_key」は上セルで設定）
result = get_cuisine_list(list_taisho_cuisine_cate_key)

KeyboardInterrupt: 

In [63]:
#取得データを表（データフレーム）に整形
df = create_df_cuisine_list(result['cuisine_list'])

128
1162


In [62]:
#エクセルに出力
#　（１）料理毎の情報（1料理で1行）
#　（２）各料理の材料_主菜・副菜」（1料理の1材料で1行）

#（１）のファイル名（拡張子も指定）　※適宜変更
export_file_name_1 = 'omame_san_1.xlsx'

#（２）のファイル名（拡張子も指定）　※適宜変更
export_file_name_2 = 'omame_san_2.xlsx'

#保存先（この jupyterファイル の格納フォルダを指定している。）
#　→フォルダを変える場合は、下記の関数「os.path.join」の第一引数を変える。
storage_path_1 = os.path.join(os.path.abspath('.') , export_file_name_1)
storage_path_2 = os.path.join(os.path.abspath('.') , export_file_name_2)

#エクセルにエクスポート
df['df_cuisine_list'].to_excel(storage_path_1)
df['df_cuisine_list_for_ingredients'].to_excel(storage_path_2)
